In [1]:
import pdfplumber

my_pdf = pdfplumber.open("codigocivilfederal.pdf")
info = my_pdf.doc.info[0]  # Access the first item in the list

decoded_info = {}
for key, value in info.items():
    if isinstance(value, bytes):
        try:
            decoded_info[key] = value.decode('utf-8')  # Try decoding with utf-8
        except UnicodeDecodeError:
            decoded_info[key] = value.decode('latin-1')  # Fallback to latin-1 if utf-8 fails
    else:
        decoded_info[key] = value

print(decoded_info)


{'Author': 'Cámara de Diputados del H. Congreso de la Unión', 'CreationDate': "D:20210122200051-06'00'", 'Creator': 'Microsoft® Office Word 2007', 'ModDate': "D:20210122200112-06'00'", 'Producer': 'Microsoft® Office Word 2007', 'Title': 'Código Civil Federal'}
